In [1]:
#Get the perpendicular distance from an edge to a Trajectory coordinate

In [2]:
import math as m

In [3]:
def NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY):
    '''
    Returns a Coord tuple of Euclidian distance from a Point to an Edge
    '''
    
    #Avoid dividing by zero
    if (EndX - StartX) == 0:
        (IntX, IntY) = (StartX, PointY)
    elif (EndY - StartY) == 0:
        (IntX, IntY) = (PointX, StartY)
    else:
        #Equation of the Edge line
        mLine = (EndY - StartY)/(EndX - StartX)
        cLine = (StartY) - (mLine*StartX)
    
        #Equation of Edge line to Trajectory point
        mLineT = (-1/mLine)
        cLineT = PointY - (mLineT*PointX)
    
        #Get Intersection of lines
        IntX = (cLine - cLineT)/(mLineT - mLine)
        IntY = (mLineT * IntX) + cLineT
    
    #This will return the closest edge coordinate if the edge is not in perpendicular range of Node
    if StartX < EndX:
        SmallerX = StartX
        LargerX = EndX
    else:
        LargerX = StartX 
        SmallerX = EndX 
    if StartY < EndY:
        SmallerY = StartY 
        LargerY = EndY
    else:
        LargerY = StartY 
        SmallerY = EndY 
    
    if SmallerX <= IntX <= LargerX and SmallerY <= IntY <= LargerY:
        return (IntX, IntY)
    else:

        NewIntX = min([SmallerX, LargerX], key=lambda x:abs(x-IntX))
        NewIntY = min([SmallerY, LargerY], key=lambda x:abs(x-IntY))
        
        return (NewIntX, NewIntY)
    

In [4]:
#GetPerpendicularDistance between an edge and a point.
def GetPerpendicularDistance(PointX, PointY, IntX, IntY):
    '''
    Returns the perpendicular distance between 2 coords
    '''
    DistanceX = PointX - IntX
    DistanceY = PointY - IntY
    Magnitude = ((DistanceX**2)+(DistanceY**2))**(1/2)
    return Magnitude


In [5]:
#GetEdgeClusters near a point
def EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, r0):
    '''
    Returns True if Edge is highlighted by a circle of radius r0 around a Point
    '''
    
    #PointX, PointY and r0 are the circle parameters, Start and End are the Edge Coordinates
    if StartX == EndX:
        if abs(r0) >= abs(StartX - PointX):
            p1 = StartX, PointY - m.sqrt(r0**2 - (StartX-PointX)**2)
            p2 = StartX, PointY + m.sqrt(r0**2 - (StartX-PointX)**2)
            inp = [p1,p2]
            # select the points lie on the line segment
            inp = [p for p in inp]
        else:
            inp = []
    else:
        k = (StartY - EndY)/(StartX - EndX)
        b0 = StartY - k*StartX
        a = k**2 + 1
        b = 2*k*(b0 - PointY) - 2*PointX
        c = (b0 - PointY)**2 + PointX**2 - r0**2
        delta = b**2 - 4*a*c
        if delta >= 0:
            p1x = (-b - m.sqrt(delta))/(2*a)
            p2x = (-b + m.sqrt(delta))/(2*a)
            p1y = k*p1x + b0
            p2y = k*p2x + b0
            inp = [[p1x,p1y],[p2x,p2y]]
            # select the points lie on the line segment
            inp = [p for p in inp]
        else:
            inp = []
           
    if inp == []:
        return False
    if inp[0][0]>=inp[1][0]:
        LargerX = inp[0][0]
        SmallerX = inp[1][0]
    else:
        LargerX = inp[1][0]
        SmallerX = inp[0][0]
    if inp[0][1]>=inp[1][1]:
        LargerY = inp[0][1]
        SmallerY = inp[1][1]
    else:
        LargerY = inp[1][1]
        SmallerY = inp[0][1]
    
    if SmallerX <= StartX <= LargerX and SmallerY <= StartY <= LargerY:
        return True
    elif SmallerX <= EndX <= LargerX and SmallerY <= EndY <= LargerY:
        return True
    else:
        return False

In [6]:
alat = 22.3
alon = 113.9185

def FilterTrajectory(Node, alat, alon, r0):
    NewNode = []
    for item in Node:
        if alat-r0 < item[0] < alat+r0 and alon-r0 < item[1] < alon+r0:
            NewNode.append(item)
    return(NewNode)

In [14]:
def Haversine(lat1, lon1, lat2, lon2):
    '''
    haversine formula which returns the distance between two Lat Lon coordinate pairs
    '''
    #Radius of the earth is in km
    R = 6372.8

    dLat = m.radians(lat2 - lat1)
    dLon = m.radians(lon2 - lon1)
    lat1 = m.radians(lat1)
    lat2 = m.radians(lat2)

    a = m.sin(dLat/2)**2 + m.cos(lat1)*m.cos(lat2)*m.sin(dLon/2)**2
    c = 2*m.asin(m.sqrt(a))

    return R * c

# Usage
lon1 = 113.92628
lat1 = 22.309341
lon2 = 113.92652
lat2 = 22.309263

print(Haversine(lat1, lon1, lat2, lon2))

0.026175739798742256


In [8]:
def EdgeSquareSearch(PointX, PointY, StartX, StartY, EndX, EndY, r0):
    '''
    Returns True if Edge is highlighted by a Square of radius r0 around a Point
    '''
    
    if PointX-r0<=StartX<=PointX+r0 and PointY-r0<=StartY<=PointY+r0:
        return True
    elif PointX-r0<=EndX<=PointX+r0 and PointY-r0<=EndY<=PointY+r0:
        return True
    else:
        return False

In [9]:
def EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY):
    '''
    This returns the percentage of an edge that is covered after a Point is snapped to it. Needs Directed Start and End
    '''
    EdgeCovered = GetPerpendicularDistance(NewStartX, NewStartY, IntX, IntY)
    EdgeLength = GetPerpendicularDistance(NewStartX, NewStartY, NewEndX, NewEndY)
    PercentageCovered = (EdgeCovered/EdgeLength)*100
    return(PercentageCovered)

In [10]:
def GetDirection1(PrevX, PrevY, IntX, IntY, StartX, StartY, EndX, EndY):
    '''Getting the direction using the distance that the snapped points point to *THERE ARE PROBLEMS WITH THIS*'''
    PrevDistance = GetPerpendicularDistance(StartX, StartY, PrevX, PrevY)
    Distance = GetPerpendicularDistance(StartX, StartY, IntX, IntY)
    if PrevDistance <= Distance:
        NewStartX = StartX
        NewStartY = StartY
        NewEndX = EndX
        NewEndY = EndY
    else:
        NewStartX = EndX
        NewStartY = EndY
        NewEndX = StartX
        NewEndY = StartY
    return (NewStartX, NewStartY, NewEndX, NewEnd)

In [11]:

#Three points involved for heading change, as the heading is derived from the next point in Trajectory
def GetHeading(StartX, StartY, EndX, EndY):
    '''Gets the angle from horizontal between two points'''
    Angle = m.atan2((EndY - StartY),(EndX - StartX))
    Degree = Angle*(180/m.pi)
    return(Degree)

def GetAlpha(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY):
    '''Gets the angle difference between Trajectory and Edge'''
    Alpha1 = GetHeading(PrevX, PrevY, PointX, PointY)
    Alpha2 = GetHeading(StartX, StartY, EndX, EndY)
    return abs(Alpha1 - Alpha2)

def GetBeta(PointX, PointY, StartX, StartY, EndX, EndY):
    '''Gets the angle between Edge and the line from Point to Junction'''
    Beta1 = GetHeading(EndX, EndY, PointX, PointY)
    Beta2 = GetHeading(EndX, EndY, StartX, StartY)
    return abs(Beta1 - Beta2)

def GetDistanceChange(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY):
    '''Gets Change in Distance between PrevPoint and Point and PrevPoint and Junction'''
    d1 = GetPerpendicularDistance(PrevX, PrevY, EndX, EndY)
    h1 = Haversine(PrevX, PrevY, EndX, EndY)
    d2 = GetPerpendicularDistance(PrevX, PrevY, PointX, PointY)
    h2 = Haversine(PrevX, PrevY, PointX, PointY)
    return (h1 - h2)

def GetHeadingIncrement(PrevX, PrevY, PointX, PointY, NextX, NextY):
    '''Gets the HeadingIncrement from previous point to current point'''
    Heading1 = GetHeading(PointX, PointY, NextX, NextY)
    Heading2 = GetHeading(PrevX, PrevY, PointX, PointY)
    return abs(Heading1 - Heading2)

In [12]:
GetAlpha(12, 29, 16, 23, 8, 30, 12, 23)

3.945186229037567

In [15]:
def GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY):
    '''This gives the direction based on the heading difference'''
    EdgeHeading = GetHeading(StartX, StartY, EndX, EndY)
    TrajHeading = GetHeading(PrevX, PrevY, PointX, PointY)
    if abs(EdgeHeading - TrajHeading) > 90:
        NewStartX = EndX
        NewStartY = EndY
        NewEndX = StartX
        NewEndY = StartY
        Direction = False
    else:
        NewStartX = StartX
        NewStartY = StartY
        NewEndX = EndX
        NewEndY = EndY
        Direction = True
        
    return (NewStartX, NewStartY, NewEndX, NewEndY, Direction)